In [0]:
ccccccccccccccccccccccccccccccccccccccccccferrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: G

In [0]:
from google.colab import files, drive

In [0]:
drive.mount('gdrive', force_remount=True)

Mounted at gdrive


In [0]:
!cp 'gdrive/My Drive/ad_clicks.zip' .

In [0]:
!unzip ad_clicks.zip

Archive:  ad_clicks.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                y

replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: train.csv               y
y
y
y



In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from datetime import *
from matplotlib import pyplot

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
train_df = pd.read_csv('train.csv')

In [0]:
train_df.shape

(12137810, 10)

In [0]:
test_df=pd.read_csv('test.csv')

In [0]:
sample_df = train_df.sample(frac=0.1, random_state = 42)
sample_df.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click
5236822,IDybLltC5,2017-01-11 10:18:32,9759547.0,525348,42227,21153737,a,NaN,Desktop,0
10696411,ID9MnifkN,2017-01-16 08:32:07,NaN,921107,904,58604466,b,Edge,Tablet,0
1823835,IDYcLRLqB,2017-01-12 10:35:50,7974132.0,707989,42227,1436904,a,Edge,Tablet,0
3295561,ID7tTKae3,2017-01-14 07:08:56,7614195.0,216666,42227,43770446,a,Mozilla,Desktop,0
6584561,IDfP3DBd0,2017-01-12 07:43:49,3428619.0,564123,50088,57977218,d,Firefox,Mobile,0


In [0]:
sample_df.shape

(1213781, 10)

In [0]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1213781 entries, 5236822 to 9230606
Data columns (total 10 columns):
ID             1213781 non-null object
datetime       1213781 non-null object
siteid         1092678 non-null float64
offerid        1213781 non-null int64
category       1213781 non-null int64
merchant       1213781 non-null int64
countrycode    1213781 non-null object
browserid      1152641 non-null object
devid          1031895 non-null object
click          1213781 non-null int64
dtypes: float64(1), int64(4), object(5)
memory usage: 101.9+ MB


In [0]:
sample_df.describe()

,siteid,offerid,category,merchant,click
count,1.092678e+06,1.213781e+06,1.213781e+06,1.213781e+06,1.213781e+06
mean,5.022356e+06,4.963481e+05,4.213800e+04,4.902424e+07,3.596695e-02
std,2.905119e+06,2.897792e+05,2.949814e+04,2.915522e+07,1.862078e-01
min,7.100000e+01,0.000000e+00,9.700000e+01,2.604430e+05,0.000000e+00
25%,2.503396e+06,2.472200e+05,1.496000e+04,2.639177e+07,0.000000e+00
50%,4.983496e+06,4.955940e+05,4.033900e+04,4.861553e+07,0.000000e+00
75%,7.584613e+06,7.470060e+05,6.678600e+04,7.436361e+07,0.000000e+00
max,9.999996e+06,9.999970e+05,9.991400e+04,9.977043e+07,1.000000e+00


In [0]:
sample_df['click'].value_counts()

0    1170125
1      43656
Name: click, dtype: int64

In [0]:
total = sample_df.isnull().sum(axis=0).sort_values(ascending=False)
percent = ((sample_df.isnull().sum(axis=0)/sample_df.isnull().count(axis=0))*100).sort_values(ascending=False)
missing_data_columns = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data_columns.head(10)

,Total,Percent
devid,181886,14.985076
siteid,121103,9.977335
browserid,61140,5.037153
click,0,0.000000
countrycode,0,0.000000
merchant,0,0.000000
category,0,0.000000
offerid,0,0.000000
datetime,0,0.000000
ID,0,0.000000


In [0]:
sample_df['siteid'].fillna(train_df['siteid'].mode()[0],inplace=True)
sample_df['browserid'].fillna(train_df['browserid'].mode()[0],inplace=True)
sample_df['devid'].fillna(train_df['devid'].mode()[0],inplace=True)

In [0]:
sample_df['datetime']=pd.to_datetime(sample_df['datetime'])
sample_df['date'] = sample_df['datetime'].dt.date
sample_df['time'] = sample_df['datetime'].dt.time
sample_df.drop(['ID','datetime'],axis= 1, inplace=True)

In [0]:
sample_df.head(5)

,siteid,offerid,category,merchant,countrycode,browserid,devid,click,date,time
5236822,9759547.0,525348,42227,21153737,a,Edge,Desktop,0,2017-01-11,10:18:32
10696411,8896401.0,921107,904,58604466,b,Edge,Tablet,0,2017-01-16,08:32:07
1823835,7974132.0,707989,42227,1436904,a,Edge,Tablet,0,2017-01-12,10:35:50
3295561,7614195.0,216666,42227,43770446,a,Mozilla,Desktop,0,2017-01-14,07:08:56
6584561,3428619.0,564123,50088,57977218,d,Firefox,Mobile,0,2017-01-12,07:43:49


In [0]:
pd.crosstab(sample_df['browserid'],sample_df['click'])

click,0,1
browserid,,
Chrome,33674,1235
Edge,403930,3064
Firefox,333098,1674
Google Chrome,55199,14507
IE,33376,1223
Internet Explorer,22293,810
InternetExplorer,55943,18685
Mozilla,110919,787
Mozilla Firefox,99684,846


In [0]:
c = {'Internet Explorer':"IntEx", 'IE':"IntEx", 'InternetExplorer':"IntEx", 'Chrome': "GoogC", 'Google Chrome': "GoogC",'Mozilla Firefox':"MF", 'Mozilla': "MF", 'Firefox': "MF", 'Opera': "Opera", 'Safari':"Safari", 'Edge':"Edge"}


In [0]:
sample_df['browserid_1']=sample_df['browserid'].map(c)

In [0]:
pd.crosstab(sample_df['devid'],sample_df['browserid_1'])

browserid_1,Edge,GoogC,IntEx,MF,Opera,Safari
devid,,,,,,
Desktop,14552,29646,63502,180526,0,0
Mobile,81371,74969,49190,366482,11395,1774
Tablet,311071,0,19638,0,0,9665


In [0]:
sample_df['browserid_encode02'] = sample_df['browserid_1'].map(sample_df.groupby('browserid_1')['click'].mean())
sample_df['devid_encode']=sample_df['devid'].map(sample_df.groupby('devid')['click'].mean())
sample_df['countrycode_encode']=sample_df['countrycode'].map(sample_df.groupby('countrycode')['click'].mean())

In [0]:
dic={'2017-01-10':'Tues','2017-01-11':'Wed','2017-01-12':'Thrus','2017-01-13':'Fri','2017-01-14':'Sat','2017-01-15':'Sun','2017-01-16':'Mon','2017-01-17':'Tues','2017-01-18':'Wed','2017-01-19':'Thrus','2017-01-20':'Fri'}

In [0]:
d=sample_df['date'].astype(str)
sample_df['day']=d.map(dic)
sample_df[sample_df['click']==1]['date'].value_counts()

2017-01-14    4705
2017-01-12    4295
2017-01-11    4217
2017-01-17    4212
2017-01-16    4103
2017-01-10    3928
2017-01-18    3925
2017-01-20    3733
2017-01-13    3693
2017-01-15    3616
2017-01-19    3229
Name: date, dtype: int64

In [0]:
def hr_func(ts):
  return ts.hour
sample_df['hour']=sample_df['time'].apply(hr_func)
sample_df[sample_df['click']==1]['hour'].value_counts()

10    3243
17    2950
16    2864
13    2827
11    2821
14    2799
12    2735
15    2660
18    2402
9     2362
21    2241
19    2162
20    2147
8     1942
22    1742
7     1445
23    1095
0      813
6      600
1      441
2      370
5      352
3      346
4      297
Name: hour, dtype: int64

In [0]:
sample_df.columns

Index(['siteid', 'offerid', 'category', 'merchant', 'countrycode', 'browserid',
       'devid', 'click', 'date', 'time', 'browserid_1', 'browserid_encode02',
       'devid_encode', 'countrycode_encode', 'day', 'hour'],
      dtype='object')

In [0]:
sample_df['dev_bro']=sample_df['devid'].astype('str')+sample_df['browserid'].astype('str')
pd.crosstab(sample_df['dev_bro'],sample_df['click'])

click,0,1
dev_bro,,
DesktopChrome,28615,1031
DesktopEdge,13600,952
DesktopInternetExplorer,47540,15962
DesktopMozilla,94460,671
DesktopMozilla Firefox,84671,724
MobileChrome,5059,204
MobileEdge,80102,1269
MobileFirefox,333098,1674
MobileGoogle Chrome,55199,14507


In [0]:
sample_df['dev_bro_encode']=sample_df['dev_bro'].map(sample_df.groupby('dev_bro')['click'].mean())
sample_df['day_encode']=sample_df['day'].map(sample_df.groupby('day')['click'].mean())

In [0]:
sample_df['site_offer']=sample_df['siteid']+sample_df['offerid']
sample_df['offer_cat']=sample_df['offerid']+sample_df['category']
sample_df['site_offer_hour']=sample_df['site_offer']+sample_df['hour']
sample_df['site_offer_dev_bro']=sample_df['site_offer']+sample_df['dev_bro_encode']
sample_df['site_offer_day']=sample_df['site_offer']+sample_df['day_encode']

In [0]:
print(sample_df.columns,len(sample_df.columns))

Index(['siteid', 'offerid', 'category', 'merchant', 'countrycode', 'browserid',
       'devid', 'click', 'date', 'time', 'browserid_1', 'browserid_encode02',
       'devid_encode', 'countrycode_encode', 'day', 'hour', 'dev_bro',
       'dev_bro_encode', 'day_encode', 'site_offer', 'offer_cat',
       'site_offer_hour', 'site_offer_dev_bro', 'site_offer_day'],
      dtype='object') 24


In [0]:
y=sample_df.click
X=sample_df.drop('click',axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [0]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
num_cols=list(X_train._get_numeric_data().columns)
fvalue_selector = SelectKBest(f_classif, k='all')
f_value=fvalue_selector.fit(X_train[num_cols],y_train)
feature=pd.DataFrame([num_cols,f_value.scores_,f_value.pvalues_])
feature

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,siteid,offerid,category,merchant,browserid_encode02,devid_encode,countrycode_encode,hour,dev_bro_encode,day_encode,site_offer,offer_cat,site_offer_hour,site_offer_dev_bro,site_offer_day
1,0.0272441,4.35194,23.4816,319.999,91864.5,12197.4,167162,2906.83,151070,312.55,0.00136226,2.51316,0.0013674,0.0013629,0.00136227
2,0.868898,0.0369671,1.26135e-06,1.49398e-71,0,0,0,0,0,6.25489e-70,0.970558,0.1129,0.970502,0.970551,0.970558


In [0]:
lr=LogisticRegression(class_weight='balanced')

In [0]:
list=['category', 'merchant','browserid_encode02', 'devid_encode','countrycode_encode', 'day_encode', 'hour','dev_bro_encode']
sample_df[list].head()

,category,merchant,browserid_encode02,devid_encode,countrycode_encode,day_encode,hour,dev_bro_encode
5236822,42227,21153737,0.007528,0.067100,0.007446,0.033931,10,0.065421
10696411,904,58604466,0.007528,0.005541,0.004016,0.036691,8,0.002710
1823835,42227,1436904,0.007528,0.005541,0.007446,0.039249,10,0.002710
3295561,42227,43770446,0.006046,0.067100,0.007446,0.033762,7,0.007053
6584561,50088,57977218,0.006046,0.038330,0.182963,0.039249,7,0.005000


In [0]:
from sklearn.preprocessing import MinMaxScaler
normalizer = MinMaxScaler()
num_columns = ['category', 'merchant']
normalizer.fit(X_train[num_columns])
X_train[num_columns] = normalizer.transform(X_train[num_columns])
X_test[num_columns] = normalizer.transform(X_test[num_columns])

In [0]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,roc_auc_score

In [0]:
def params(yt,yp):
  print('Accuracy:',accuracy_score(yt,yp))
  print('F1score:',f1_score(yt,yp))
  print('Prec:',precision_score(yt,yp))
  print('Recall',recall_score(yt,yp))
  print('ROC:',roc_auc_score(yt,yp))

In [0]:
lr.fit(X_train[list],y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [0]:
y_pred=lr.predict(X_test[list])
y_train_pred=lr.predict(X_train[list])

In [0]:
print('Testing params')
params(y_test,y_pred)

Testing params
Accuracy: 0.8641492852925426
F1score: 0.32450294953026004
Prec: 0.19752963276977042
Recall 0.9084792415322273
ROC: 0.8854884813467617


In [0]:
print('Training params')
params(y_train,y_train_pred)

Training params
Accuracy: 0.8637879775812515
F1score: 0.3249337953079247
Prec: 0.19773393629180538
Recall 0.9109134316643228
ROC: 0.8864710748177902


In [0]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(class_weight='balanced',max_depth=4,n_estimators=100)
rfc.fit(X_train[list],y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=4, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [0]:
y_pred_rf=rfc.predict(X_test[list])
y_train_pred_rf=rfc.predict(X_train[list])

In [0]:
print('Test')
params(y_test,y_pred_rf)

Test
Accuracy: 0.8830131681931151
F1score: 0.35710297158207693
Prec: 0.2224624873077357
Recall 0.9045798608456304
ROC: 0.8933947683973834


In [0]:
print('Test')
params(y_test,y_pred_rf)

Test
Accuracy: 0.8830131681931151
F1score: 0.35710297158207693
Prec: 0.2224624873077357
Recall 0.9045798608456304
ROC: 0.8933947683973834


In [0]:
from sklearn.metrics import classification_report

In [0]:
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      0.88      0.94    351056
           1       0.22      0.90      0.36     13079

   micro avg       0.88      0.88      0.88    364135
   macro avg       0.61      0.89      0.65    364135
weighted avg       0.97      0.88      0.91    364135



In [0]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train[list], y_train)
y_pred_xg = model.predict(X_test[list])
y_train_xg = model.predict(X_train[list])

In [0]:
print('Test')
params(y_test,y_pred_xg)

Test
Accuracy: 0.9785134634132945
F1score: 0.6876646706586825
Prec: 0.7194887645142427
Recall 0.6585365853658537
ROC: 0.8244855799533339


In [0]:
print('Train')
params(y_train,y_train_xg)

Train
Accuracy: 0.9784616181327283
F1score: 0.6883833395770188
Prec: 0.7180716899356994
Recall 0.6610524250253459
ROC: 0.8256816878145096


In [0]:
print(classification_report(y_test,y_pred_xg))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    351056
           1       0.72      0.66      0.69     13079

   micro avg       0.98      0.98      0.98    364135
   macro avg       0.85      0.82      0.84    364135
weighted avg       0.98      0.98      0.98    364135

